In [52]:
import random
from faker import Faker
from pymongo import MongoClient

In [53]:
fake = Faker()

In [54]:
MONGO_CONNECTION_STRING = "mongodb://localhost:27017/"
DB_NAME = "hospital_db"
COLLECTION_NAME = "data"

In [55]:
try:
    client = MongoClient(MONGO_CONNECTION_STRING)
    # Renaming the connection variable as requested in the prompt
    current_weather = client[DB_NAME][COLLECTION_NAME]
    current_weather.drop()  # Clear previous data for a fresh start
    print(f"Successfully connected to MongoDB and selected database: {DB_NAME}, collection: {COLLECTION_NAME}.")
except Exception as e:
    print(f"Error connecting to MongoDB: {e}")
    current_weather = None

Successfully connected to MongoDB and selected database: hospital_db, collection: data.


In [56]:
DEPARTMENTS = ["Cardiology", "Neurology", "Oncology", "Pediatrics", "Emergency"]
DOCTOR_ROLES = ["Attending Physician", "Resident", "Surgeon", "Specialist"]
PATIENT_CONDITIONS = ["Flu", "Broken Bone", "Heart Disease", "Migraine", "Asthma", "Diabetes"]
APPOINTMENT_STATUS = ["Scheduled", "Completed", "Canceled"]
MEDICAL_RECORD_TYPES = ["Admission Summary", "Discharge Notes", "Lab Results", "Prescription"]

In [57]:
if current_weather is not None:
    NUM_RECORDS = 10
    print(f"Starting insertion of {NUM_RECORDS} synthetic hospital records...")
    
    records_to_insert = []
    
    # Generate data using a for loop
    for i in range(NUM_RECORDS):
        
        # 1. Patient Data
        patient_name = fake.name()
        patient_id = i + 1001
        
        # 2. Doctor Data
        doctor_name = fake.name()
        department = random.choice(DEPARTMENTS)
        
        # 3. Appointment/Record Data
        appointment_date = fake.date_time_between(start_date="-1y", end_date="now")
        record_type = random.choice(MEDICAL_RECORD_TYPES)
        
        # Create a single, nested document for the 'data' collection
        hospital_document = {
            "record_id": i + 1,
            "entity_type": random.choice(["patient_record", "appointment"]),
            "patient": {
                "patient_id": patient_id,
                "name": patient_name,
                "dob": fake.date_of_birth(minimum_age=1, maximum_age=90).isoformat(),
                "contact": fake.phone_number(),
                "condition": random.choice(PATIENT_CONDITIONS)
            },
            "doctor": {
                "doctor_name": doctor_name,
                "role": random.choice(DOCTOR_ROLES),
                "department": department
            },
            "appointment": {
                "date": appointment_date.isoformat(),
                "status": random.choice(APPOINTMENT_STATUS),
                "reason": f"Consultation for {random.choice(PATIENT_CONDITIONS)}"
            },
            "medical_record": {
                "type": record_type,
                "content": fake.text(max_nb_chars=100)
            }
        }
        
        records_to_insert.append(hospital_document)

    # Insert the generated records in bulk
    if records_to_insert:
        insert_result = current_weather.insert_many(records_to_insert)
        print(f"\nData insertion complete. Inserted {len(insert_result.inserted_ids)} records.")

        # --- Verification Read Operation ---
        print("\n--- Verifying First 3 Inserted Records ---")
        for record in current_weather.find().limit(3):
            # Using pprint for cleaner dictionary output
            import pprint
            pprint.pprint(record)

else:
    print("Cannot proceed with data insertion due to MongoDB connection error (current_weather is None).")

Starting insertion of 10 synthetic hospital records...

Data insertion complete. Inserted 10 records.

--- Verifying First 3 Inserted Records ---
{'_id': ObjectId('68ff5a12f915a7f4c6884385'),
 'appointment': {'date': '2025-08-05T10:37:39',
                 'reason': 'Consultation for Broken Bone',
                 'status': 'Canceled'},
 'doctor': {'department': 'Neurology',
            'doctor_name': 'Michelle Cook',
            'role': 'Surgeon'},
 'entity_type': 'patient_record',
 'medical_record': {'content': 'Fish have ten. Thing this street throw yes '
                               'raise. Resource social small open list.',
                    'type': 'Discharge Notes'},
 'patient': {'condition': 'Diabetes',
             'contact': '(377)306-3065x103',
             'dob': '1957-11-09',
             'name': 'Kyle Williams',
             'patient_id': 1001},
 'record_id': 1}
{'_id': ObjectId('68ff5a12f915a7f4c6884386'),
 'appointment': {'date': '2025-09-06T23:42:38',
             

In [58]:
def create_appointment(patient_id, doctor_name, date, reason):
    """Creates a new appointment record with a consistent 'manual_id' for CRUD testing."""
    if current_weather is not None: 
        appointment = {
            "entity_type": "appointment",
            "patient_id": patient_id,
            "doctor_name": doctor_name,
            "date": date,
            "reason": reason,
            "status": "Scheduled",
            # This is the key we will use for update/delete later
            "manual_id": 999 
        }
        result = current_weather.insert_one(appointment)
        print(f" CREATE: Appointment created with MongoDB ID: {result.inserted_id}")

def read_patient_record(patient_id):
    """Reads all records associated with a specific patient ID."""
    if current_weather is not None:
        query = {"$or": [{"patient.patient_id": patient_id}, {"patient_id": patient_id}]}
        records = list(current_weather.find(query))
        return records
    return []

def update_appointment_status(manual_id, new_status):
    """Updates the status of a specific appointment record using the 'manual_id' field."""
    if current_weather is not None:
        # Parameter name matches the call: manual_id=999
        query = {"manual_id": manual_id, "entity_type": "appointment"}
        update_operation = {"$set": {"appointment.status": new_status}}
        result = current_weather.update_one(query, update_operation)
        print(f" UPDATE: Matched {result.matched_count} record(s), Modified {result.modified_count} record(s) for ID {manual_id}.")

def delete_record(manual_id):
    """Permanently deletes a record by its custom 'manual_id' field."""
    if current_weather is not None:
        # Parameter name matches the call: manual_id=999
        query = {"manual_id": manual_id}
        result = current_weather.delete_one(query)
        print(f" DELETE: Deleted {result.deleted_count} record(s) for manual_id {manual_id}.")

In [60]:
from bson.objectid import ObjectId
import pprint

TARGET_OBJECT_ID = '68ff598ef915a7f4c6884379'

# --- Example Usage of CRUD Functions ---

if 'current_weather' in locals() and current_weather is not None:
    
    # 1. CREATE OPERATION (Used to create a record with manual_id=999 for update/read demonstration)
    print("\n--- 1. CREATE OPERATION ---")
    create_appointment(
        patient_id=1005, 
        doctor_name="Dr. Smith", 
        date="2025-11-10T10:00:00", 
        reason="Follow-up on Diabetes"
    )

    # 2. UPDATE OPERATION
    print("\n--- 2. UPDATE OPERATION ---")
    update_appointment_status(manual_id=999, new_status="Completed")

    # 3. READ OPERATION: Patient 1005 Records
    print("\n--- 3. READ OPERATION: Patient 1005 Records ---")
    patient_records = read_patient_record(1005) 
    print(f"Found {len(patient_records)} records for Patient 1005.")
    if patient_records:
        print("Latest Record (Verify 'Completed' Status):")
        pprint.pprint(patient_records[-1])

    # 4. DELETE OPERATION: Delete the specific document from the image by its _id
    print("\n--- 4. DELETE SPECIFIC DOCUMENT BY _ID ---")
    
    try:
        # This directly uses the MongoDB method to delete the specific document by its unique _id.
        result = current_weather.delete_one({"_id": ObjectId(TARGET_OBJECT_ID)})
        
        if result.deleted_count == 1:
            print(f" SUCCESS: Document with _id {TARGET_OBJECT_ID} has been removed.")
        else:
            print(f" WARNING: No document found with _id {TARGET_OBJECT_ID}. (It might have already been deleted)")

    except Exception as e:
        print(f" ERROR deleting document: {e}")

    # 5. VERIFY DELETION (The original 'manual_id=999' verification is removed, 
    # as we now check the document from the image.)
    print("\n--- 5. VERIFY DELETION ---")
    deleted_record_check = current_weather.find_one({"_id": ObjectId(TARGET_OBJECT_ID)})
    if deleted_record_check is None:
        print("Verification successful: The target document is removed.")
    else:
        print("Verification failed: The target document still exists.")
    
    
    if 'client' in locals():
        # client.close() 
        print("\nNote: Remember to close the MongoDB client connection ('client.close()') after all operations are complete.")


--- 1. CREATE OPERATION ---
 CREATE: Appointment created with MongoDB ID: 68ff5a1cf915a7f4c6884390

--- 2. UPDATE OPERATION ---
 UPDATE: Matched 1 record(s), Modified 0 record(s) for ID 999.

--- 3. READ OPERATION: Patient 1005 Records ---
Found 3 records for Patient 1005.
Latest Record (Verify 'Completed' Status):
{'_id': ObjectId('68ff5a1cf915a7f4c6884390'),
 'date': '2025-11-10T10:00:00',
 'doctor_name': 'Dr. Smith',
 'entity_type': 'appointment',
 'manual_id': 999,
 'patient_id': 1005,
 'reason': 'Follow-up on Diabetes',
 'status': 'Scheduled'}

--- 4. DELETE SPECIFIC DOCUMENT BY _ID ---

--- 5. VERIFY DELETION ---
Verification successful: The target document is removed.

Note: Remember to close the MongoDB client connection ('client.close()') after all operations are complete.
